# Rapport du Projet : Segmentation de vaisseaux sur des images de la rétine

## Ossee Yiboe
### Mars 2023


## 1.Problématique et but du projet

L’ophtalmoscopie à balayage laser (dite SLO, pour scanning laser ophthalmoscopy) est une modalité d’imagerie de la rétine permettant de réaliser un fond d’œil à grande résolution et avec un large champ, et donc d’observer sur une seule image la majeure partie de la surface de la rétine à une résolution entre 10 et 100 µm.

Outre les maladies de la rétine elle-même, l’observation du fond d’œil permet de diagnostiquer plusieurs pathologies générales en observant la circulation artérielle et veineuse dans la rétine. C’est le cas en particulier de l’hypertension artérielle et de l’insuffisance rénale. Le diagnostic repose en général sur une analyse quantitative de l’ensemble du réseau vasculaire de l’image de rétine, et nécessite donc une segmentation précise de ce réseau.

Le but de ce projet est de proposer une méthode automatique de segmentation du réseau vasculaire dans des images de rétine SLO. La figure 1 montre deux exemples d’images SLO, ainsi que les images de vérité terrain (Ground Truth), correspondant aux annotations manuelles d’un expert.

## 2.Données et outils informatiques


Les données - images et annotations - sont extraites de la base de données IOSTAR de l’IDIAP pour l’évaluation d’algorithmes de segmentation de réseau vasculaire rétinien :

https://www.idiap.ch/software/bob/docs/bob/bob.db.iostar/stable/

On ne vous demande pas de télécharger toute la base, mais d’utiliser la sélection fournie dans l’archive suivante :

https://perso.ensta-paris.fr/~manzaner/Cours/MI206/images_tp2.zip

Le TP-projet peut être réalisé entièrement en Python. Vous pouvez réutiliser les fonctions créées au TP1, mais vous devrez intégrer votre algorithme de segmentation au script de base suivant, qui fournit une fonction d’évaluation pour mesurer la qualité de votre segmentation :

https://perso.ensta-paris.fr/~manzaner/Cours/MI206/script_tp2.py

Vous pouvez aussi développer et coder votre algorithme de segmentation en utilisant Inti, et n’utiliser le script Python que pour évaluer votre algorithme.

In [7]:
import cv2
import math
import numpy as np
from PIL import Image
from scipy import ndimage as ndi
from skimage import data, filters
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte
from matplotlib import pyplot as plt
from skimage import io, filters, feature, segmentation, morphology
from skimage.morphology import square, diamond, octagon, rectangle, star, disk
from skimage.morphology import erosion, dilation, binary_erosion, opening, closing, white_tophat, reconstruction, black_tophat, skeletonize, convex_hull_image, thin



def evaluate(img_out, img_GT):
    GT_skel  = thin(img_GT, max_num_iter = 15) # On suppose que la demie epaisseur maximum 
    img_out_skel  = thin(img_out, max_num_iter = 15) # d'un vaisseau est de 15 pixels...
    TP = np.sum(img_out_skel & img_GT) # Vrais positifs
    FP = np.sum(img_out_skel & ~img_GT) # Faux positifs
    FN = np.sum(GT_skel & ~img_out) # Faux negatifs

    ACCU = TP / (TP + FP) # Precision
    RECALL = TP / (TP + FN) # Rappel
    return ACCU, RECALL, img_out_skel, GT_skel




## Implémentation

La procédure utilisée pour ce travail de segmentation est détaillée dans les lignes qui suivent.



- Nous commençons par charger l'image à partir du chemin spécifié, puis la convertir en échelle de gris en utilisant la fonction cv2.cvtColor() de la bibliothèque OpenCV et la classe Image de la bibliothèque PIL (Python Imaging Library).

- Nous effectuons une transformation black tophat en utilisant la fonction $black\_tophat()$ de la bibliothèque skimage.morphology. La transformation black tophat est une technique de traitement d'image qui permet de mettre en évidence les structures de faible contraste dans l'image en soustrayant une version fermée de l'image originale.

- Ensuite nous réalisons une opération de fermeture sur l'image transformée en utilisant la fonction closing() de la bibliothèque skimage.morphology. L'opération de fermeture est une technique de traitement d'image qui permet de remplir les petits trous et de lisser les contours des objets dans l'image.

- Puis nous appliquons le filtre de Meijering en utilisant la fonction meijering() de la bibliothèque skimage.filters. Le filtre de Meijering est une technique de traitement d'image qui permet de détecter les ridules sombres et claires dans l'image. L'étape suivante est l'application d'un seuillage en utilisant un seuil de 0,17 pour obtenir les vaisseaux sanguins.

- L'avant dernière étape consiste à appliquer un filtre Gaussien en utilisant la fonction gaussian() de la bibliothèque skimage.filters pour améliorer la précision de la segmentation. 

- Finalement, nous appliquons une opération de dilatation en utilisant la fonction dilation() de la bibliothèque skimage.morphology pour connecter les vaisseaux sanguins et combler les petits espaces.

- En fin de compte, l'image résultante est convertie en booléen pour l'évaluation de la précision et du rappel et est comparée à l'image ground truth pour évaluer les performances de la fonction de segmentation. Nous convertissons ensuite l'image en un tableau booléen à l'aide de la méthode astype(). Le reste de la manipulation consiste et l'évaluation et la représentation graphiques des résultats dont les étapes sont les suivantes : chargement de l'image de vérité terrain à partir du chemin spécifié, puis la convertir en un tableau booléen, évaluation de  l'exactitude et le rappel de la segmentation en utilisant la fonction evaluate(), et affichage de l'image finale et retour de l'image segmentée, la précision et le rappel.


In [8]:
path = './images_IOSTAR/'

In [69]:
def my_segmentation (image_path, test_path) :

    # Load the image and convert it to grayscale
    img = cv2.imread(image_path, cv2.IMREAD_COLOR) 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(img.astype(np.uint8))
    gray_image = img_as_float(color.rgb2gray(image))

    # Perform black tophat transformation
    image_hat = black_tophat(gray_image, footprint=disk(5.5)) 

    # Perform closing operation
    image_closing = closing(image_hat, footprint=disk(3))

    # Apply Meijering filter and thresholding
    final_filter = meijering(image_closing, black_ridges=None, sigmas=range(1, 5))
    vessels = final_filter > 0.17

    # Apply Gaussian filter and dilation operation
    vessels = filters.gaussian(vessels, sigma=0.2) 
    image_final = dilation(vessels, footprint=disk(2)) 

    # Convert to boolean
    img_out = image_final.astype(np.bool_)

    # Load ground truth image and convert to boolean
    img_GT = np.asarray(Image.open(test_path)).astype(np.bool_)

    # Evaluate accuracy and recall
    ACCU, RECALL, img_out_skel, GT_skel = evaluate(img_out, img_GT)
    #print('Accuracy =', ACCU, ', Recall =', RECALL)

    return img_out, ACCU, RECALL

In [72]:
liste_to_eval = ['GT_01.png', 'GT_02.png', 'GT_03.png', 'GT_08.png' , 'GT_21.png', 'GT_26.png', 'GT_28.png', 'GT_32.png' ,  'GT_37.png', 'GT_48.png' ]
liste_to_seg = ['star01_OSC.jpg', 'star02_OSC.jpg', 'star03_OSN.jpg', 'star08_OSN.jpg', 'star21_OSC.jpg', 'star26_ODC.jpg', 'star28_ODN.jpg', 'star32_ODC.jpg', 'star37_ODN.jpg', 'star48_OSN.jpg' ] 


results_df = pd.DataFrame() #columns=['Filename', 'Accuracy', 'Recall']

names = []
accs =[]
recs = []


for i in range(len(liste_to_eval)) :
    
    img_out, ACCU, RECALL = my_segmentation (path + liste_to_seg[i], path + liste_to_eval[i])
    
    # append the results to the DataFrame
    names.append(liste_to_seg[i])
    accs.append(ACCU)
    recs.append(RECALL)
    
names = np.array(names)
accs = np.array(accs)
recs = np.array(recs)
print("Les résultats obtenus sont les suivants : \n")
print("Average accuracy",np.mean(accs))
print("Average recall",np.mean(recs),"\n")


results_df['Filename'] = pd.Series(names) 
results_df['Accuracy'] = pd.Series(accs)    
results_df['Recall'] = pd.Series(recs)    

results_df

Les résultats obtenus sont les suivants : 

Average accuracy 0.7748870805985271
Average recall 0.7342559797324668 



,Filename,Accuracy,Recall
0,star01_OSC.jpg,0.761088,0.784559
1,star02_OSC.jpg,0.772534,0.812873
2,star03_OSN.jpg,0.756294,0.729088
3,star08_OSN.jpg,0.815210,0.760514
4,star21_OSC.jpg,0.711346,0.685463
5,star26_ODC.jpg,0.809146,0.632731
6,star28_ODN.jpg,0.775461,0.599926
7,star32_ODC.jpg,0.777452,0.797383
8,star37_ODN.jpg,0.789119,0.734950
9,star48_OSN.jpg,0.781222,0.805073


## Optimisation des performances

- La phase finale de notre procédure utilise une approche de recherche exhaustive pour trouver les meilleurs paramètres pour différents algorithmes de traitement d'images tels que la transformation black tophat, la fermeture, le filtre de Meijering, la dilatation et l'application du filtre gaussien. Finalement, les meilleurs paramètres pour la précision et le rappel sont utilisés pour la segmentation. 

In [44]:
def fine_tune(image_path, test_path) :
    
    # Load the image and convert it to grayscale
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(img.astype(np.uint8))
    gray_image = img_as_float(color.rgb2gray(image))

    # Load ground truth image and convert to boolean
    img_GT = np.asarray(Image.open(test_path)).astype(np.bool_)

    best_accu = 0
    best_recall = 0
    best_disk_size_accu = 0
    best_threshold_accu = 0
    best_disk_size_recall = 0
    best_threshold_recall = 0
    best_sigma = 0
    best_disk_size_accu_bh = 0
    best_disk_size_accu_dil = 0
    best_disk_size_accu_clo= 0
    best_sigma_accu_bh = 0
    best_sigma_accu_dil = 0
    best_disk_size_recall_bh = 0
    best_disk_size_recall_dil = 0
    best_disk_size_recall_clo = 0
    best_sigma_recall_bh = 0
    best_sigma_recall_dil = 0

    for disk_size_bh in range(3, 10):
        for disk_size_dil in range(1, 4):
            for disk_size_clo in range(1, 4):
                for sigma in np.arange(0.1, 1, 0.1):
                    for threshold in np.arange(0.1, 0.3, 0.01):
                        # Perform black tophat transformation
                        image_hat = black_tophat(gray_image, footprint=disk(disk_size_bh))

                        # Perform closing operation
                        image_closing = closing(image_hat, footprint=disk(disk_size_clo))

                        # Apply Meijering filter and thresholding
                        final_filter = meijering(image_closing, black_ridges=None, sigmas=range(1, 5))
                        vessels = final_filter > threshold

                        # Apply Gaussian filter and dilation operation
                        vessels = filters.gaussian(vessels, sigma=sigma)
                        image_final = dilation(vessels, footprint=disk(disk_size_dil))  #1

                        # Convert to boolean
                        img_out = image_final.astype(np.bool_)

                        # Evaluate accuracy and recall
                        ACCU, RECALL, img_out_skel, GT_skel = evaluate(img_out, img_GT)

                        # Update best parameters
                        if (ACCU + RECALL) / 2 > (best_accu + best_recall) / 2:
                            best_accu = ACCU
                            best_recall = RECALL
                            best_disk_size_accu = disk_size_bh
                            best_threshold_accu = threshold
                            best_disk_size_recall = disk_size_bh
                            best_threshold_recall = threshold
                            best_sigma = sigma
                            best_disk_size_accu_bh = disk_size_bh
                            best_disk_size_accu_dil = disk_size_dil
                            best_disk_size_accu_clo = disk_size_clo
                            best_sigma_accu_bh = sigma
                            best_sigma_accu_dil = sigma
                            best_disk_size_recall_bh = disk_size_bh
                            best_disk_size_recall_dil = disk_size_dil
                            best_disk_size_recall_clo = disk_size_clo
                            best_sigma_recall_bh = sigma
                            best_sigma_recall_dil = sigma

    print("Disk size for dilatation: ", best_disk_size_accu_dil)
    print("Disk size for closing: ", best_disk_size_accu_clo)
    print("Sigma value for Gaussian denoising: ", best_sigma)
    print("Threshold for Meijering filter: ", best_threshold_accu)
    print("Accuracy: ", best_accu)
    print("Recall: ", best_recall)
    print("================")
    print("Best combination for recall and accuracy:")
    print("Disk size for black tophat erosion: ", best_disk_size_recall_bh)
    print("Disk size for dilation: ", best_disk_size_recall_dil)
    print("Disk size for closing: ", best_disk_size_recall_clo)
    print("Sigma value for Gaussian denoising: ", best_sigma_recall_bh)
    print("Threshold for Meijering filter: ", best_threshold_recall)
    print("Recall: ", best_recall)
    print("Accuracy: ", best_accu)
    
    return best_recall, best_accu


results_df = pd.DataFrame() 

names = []
accs =[]
recs = []


for i in range(len(liste_to_eval)) :
    print(liste_to_seg[i])
    best_recall, best_accu = fine_tune(path + liste_to_seg[i], path + liste_to_eval[i])
    
    # append the results to the DataFrame
    names.append(liste_to_seg[i])
    accs.append(best_accu)
    recs.append(best_recall)
    
names = np.array(names)
accs = np.array(accs)
recs = np.array(recs)

print("Average accuracy",np.mean(accs),"\n")
print("Average recall",np.mean(recs),"\n")


results_df['Filename'] = pd.Series(names) 
results_df['Accuracy'] = pd.Series(accs)    
results_df['Recall'] = pd.Series(recs)    

results_df



star01_OSC.jpg
Disk size for dilatation:  2
Disk size for closing:  3
Sigma value for Gaussian denoising:  0.2
Threshold for Meijering filter:  0.16999999999999998
Accuracy:  0.7527789213668176
Recall:  0.788146551724138
Best combination for recall and accuracy:
Disk size for black tophat erosion:  5
Disk size for dilation:  2
Disk size for closing:  3
Sigma value for Gaussian denoising:  0.2
Threshold for Meijering filter:  0.16999999999999998
Recall:  0.788146551724138
Accuracy:  0.7527789213668176
star02_OSC.jpg
Disk size for dilatation:  1
Disk size for closing:  3
Sigma value for Gaussian denoising:  0.2
Threshold for Meijering filter:  0.15999999999999998
Accuracy:  0.7288015577978403
Recall:  0.8395187601957586
Best combination for recall and accuracy:
Disk size for black tophat erosion:  9
Disk size for dilation:  1
Disk size for closing:  3
Sigma value for Gaussian denoising:  0.2
Threshold for Meijering filter:  0.15999999999999998
Recall:  0.8395187601957586
Accuracy:  0.728

,Filename,Accuracy,Recall
0,star01_OSC.jpg,0.752779,0.788147
1,star02_OSC.jpg,0.728802,0.839519
2,star03_OSN.jpg,0.777354,0.714023
3,star08_OSN.jpg,0.832318,0.725258
4,star21_OSC.jpg,0.732635,0.657704
5,star26_ODC.jpg,0.656066,0.785157
6,star28_ODN.jpg,0.676616,0.726452
7,star32_ODC.jpg,0.735366,0.823770
8,star37_ODN.jpg,0.765595,0.739266
9,star48_OSN.jpg,0.855755,0.713503
